<a href="https://colab.research.google.com/github/sandroormeno/TensorFlow-LITE-ESP32-ARDUINO/blob/master/tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow==2.1.0

In [0]:
%tensorflow_version 2.x

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pandas
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from keras.utils import to_categorical, np_utils
from sklearn.model_selection import train_test_split
from keras import backend as K

In [30]:
print("TensorFlow version: {}".format(tf.__version__))

TensorFlow version: 2.1.0


In [31]:
url = "https://raw.githubusercontent.com/sandroormeno/Tensorflow-basic-examples/master/datos/iris.csv"
datosIris = pandas.read_csv(url)
datosIris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
datos=datosIris.values[:,:4]
etiquetas=datosIris.values[:,4]

In [0]:
X_train,X_test,y_train,y_test=train_test_split(datos,etiquetas,train_size=0.8,random_state=23)

In [0]:
def one_hot(arr):
    uniques, ids = np.unique(arr, return_inverse=True)
    return np_utils.to_categorical(ids, len(uniques))
y_train_ohe=one_hot(y_train)
y_test_ohe=one_hot(y_test)

In [0]:
# los datos de entrada dan un error, al no poder convertir Float en tensores.
# soluciòn aquì : https://blog.csdn.net/weixin_44473755/article/details/102939703
X_train_1 = K.cast_to_floatx (X_train)
X_test_1 = K.cast_to_floatx (X_test)

In [36]:
print (X_train_1[20] )

[7.4 2.8 6.1 1.9]


In [0]:
# El modelo
model= tf.keras.Sequential()

model.add(layers.Dense(16, activation='relu', input_dim=4))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(3, activation='sigmoid'))

# Configulación del entrenamiento
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 16)                80        
_________________________________________________________________
dense_4 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 51        
Total params: 675
Trainable params: 675
Non-trainable params: 0
_________________________________________________________________


In [39]:
model.fit(X_train_1,y_train_ohe, epochs=100,batch_size=1,verbose=1)

Train on 120 samples
Epoch 1/100
120/120 [==============================] - 1s 4ms/sample - loss: 1.0723 - accuracy: 0.3083
Epoch 2/100
120/120 [==============================] - 0s 2ms/sample - loss: 0.9582 - accuracy: 0.4333
Epoch 3/100
120/120 [==============================] - 0s 2ms/sample - loss: 0.6447 - accuracy: 0.7500
Epoch 4/100
120/120 [==============================] - 0s 2ms/sample - loss: 0.4341 - accuracy: 0.8167
Epoch 5/100
120/120 [==============================] - 0s 2ms/sample - loss: 0.3265 - accuracy: 0.8917
Epoch 6/100
120/120 [==============================] - 0s 2ms/sample - loss: 0.2544 - accuracy: 0.9167
Epoch 7/100
120/120 [==============================] - 0s 2ms/sample - loss: 0.1914 - accuracy: 0.9583
Epoch 8/100
120/120 [==============================] - 0s 1ms/sample - loss: 0.1662 - accuracy: 0.9500
Epoch 9/100
120/120 [==============================] - 0s 2ms/sample - loss: 0.1632 - accuracy: 0.9417
Epoch 10/100
120/120 [==============================

In [40]:
loss, accuracy = model.evaluate(X_test_1, y_test_ohe, verbose=0)
print("Test accuracy : " + str(accuracy))

Test accuracy : 0.96666664


In [41]:
# Stay here
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
tflite_model = converter.convert()

# Save the model to disk
open("exercice_model.tflite", "wb").write(tflite_model)

4624

In [42]:
X_test = X_test.astype('float32')

# Instantiate an interpreter for each model
exercice_model = tf.lite.Interpreter('exercice_model.tflite')

# Allocate memory for each model
exercice_model.allocate_tensors()

# Get the input and output tensors so we can feed in values and get the results
input_details = exercice_model.get_input_details()
output_details = exercice_model.get_output_details()

test = 15

input_data = X_test_1[test].reshape((1,4))
exercice_model.set_tensor(input_details[0]['index'],input_data )

exercice_model.invoke()
  
output_data = exercice_model.get_tensor(output_details[0]['index'])
    
  
print(output_data)  
print(y_test[test])

[[4.5722413e-09 8.4328027e-03 5.2129449e-03]]
Iris-virginica


In [43]:
import os
basic_model_size = os.path.getsize("exercice_model.tflite")
print("Basic model is %d bytes" % basic_model_size)

Basic model is 4624 bytes


In [0]:
# Install xxd if it is not available
!sudo apt-get update
!sudo apt-get install xxd

In [45]:
# Save the file as a C source file
!xxd -i exercice_model.tflite > exercice_model.cc
# Print the source file
!cat exercice_model.cc

unsigned char exercice_model_tflite[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x00, 0x00, 0x12, 0x00,
  0x1c, 0x00, 0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00,
  0x00, 0x00, 0x18, 0x00, 0x12, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0xbc, 0x11, 0x00, 0x00, 0xb8, 0x0b, 0x00, 0x00, 0xa0, 0x0b, 0x00, 0x00,
  0x34, 0x00, 0x00, 0x00, 0x04, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00,
  0x04, 0x00, 0x00, 0x00, 0xec, 0xee, 0xff, 0xff, 0x08, 0x00, 0x00, 0x00,
  0x0f, 0x00, 0x00, 0x00, 0x13, 0x00, 0x00, 0x00, 0x6d, 0x69, 0x6e, 0x5f,
  0x72, 0x75, 0x6e, 0x74, 0x69, 0x6d, 0x65, 0x5f, 0x76, 0x65, 0x72, 0x73,
  0x69, 0x6f, 0x6e, 0x00, 0x10, 0x00, 0x00, 0x00, 0x60, 0x0b, 0x00, 0x00,
  0x44, 0x0b, 0x00, 0x00, 0x6c, 0x0a, 0x00, 0x00, 0x1c, 0x0a, 0x00, 0x00,
  0x0c, 0x06, 0x00, 0x00, 0xfc, 0x04, 0x00, 0x00, 0xf4, 0x04, 0x00, 0x00,
  0xa4, 0x04, 0x00, 0x00, 0x9c, 0x04, 0x00, 0x00, 0x4c, 0x04, 0x00, 0x00,
  0x44, 0x04, 0x00, 0x00, 0x3c, 0x04, 0x00, 0x00, 0x34, 0x04, 0x00, 0x

In [0]:
for i in range(len(X_test_1)):
  salida = str(X_test_1[i]) + str(y_test_ohe[i])
  print(salida)

[6.9 3.1 5.4 2.1][0. 0. 1.]
[7.2 3.2 6.  1.8][0. 0. 1.]
[5.8 2.7 4.1 1. ][0. 1. 0.]
[4.6 3.1 1.5 0.2][1. 0. 0.]
[5.7 2.5 5.  2. ][0. 0. 1.]
[6.9 3.1 4.9 1.5][0. 1. 0.]
[5.7 3.8 1.7 0.3][1. 0. 0.]
[6.4 3.1 5.5 1.8][0. 0. 1.]
[5.4 3.4 1.7 0.2][1. 0. 0.]
[5.6 2.9 3.6 1.3][0. 1. 0.]
[6.1 2.8 4.7 1.2][0. 1. 0.]
[4.8 3.1 1.6 0.2][1. 0. 0.]
[7.1 3.  5.9 2.1][0. 0. 1.]
[5.1 3.8 1.5 0.3][1. 0. 0.]
[5.4 3.7 1.5 0.2][1. 0. 0.]
[6.3 2.8 5.1 1.5][0. 0. 1.]
[6.4 3.2 4.5 1.5][0. 1. 0.]
[6.1 2.9 4.7 1.4][0. 1. 0.]
[7.7 3.  6.1 2.3][0. 0. 1.]
[5.2 3.4 1.4 0.2][1. 0. 0.]
[6.4 3.2 5.3 2.3][0. 0. 1.]
[4.8 3.  1.4 0.3][1. 0. 0.]
[5.1 3.8 1.6 0.2][1. 0. 0.]
[4.7 3.2 1.3 0.2][1. 0. 0.]
[6.7 3.1 5.6 2.4][0. 0. 1.]
[5.1 3.5 1.4 0.3][1. 0. 0.]
[5.1 3.8 1.9 0.4][1. 0. 0.]
[6.2 2.8 4.8 1.8][0. 0. 1.]
[6.6 3.  4.4 1.4][0. 1. 0.]
[5.6 2.5 3.9 1.1][0. 1. 0.]


In [0]:
print(X_test_1.shape)

(30, 4)


In [0]:
print(y_test_ohe)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]
